# Thermal-Hydraulic Circuit Modeling

In [1]:
from multiprocessing.dummy import freeze_support
from system import *
import os
import pandas as pd
import csv
import codecs
from CoolProp.CoolProp import PropsSI
import ipywidgets as wd
from IPython.display import display

In [2]:
# generates JPCM from .csv-file
directory = os.getcwd()
data = pd.read_csv(directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/JPCM.csv', sep=';', encoding='utf-8-sig',
                   header=None)
jpcm = data.loc[1:, 1:].to_numpy().astype(int)

# defines component types of the cycle
component_type_list = ['Compressor',
                       'Condenser',
                       'Separator',
                       'Expansion Valve',
                       'Evaporator',
                       'Pump',
                       'Heat Exchanger',
                       'Mixing Valve',
                       'Pump',
                       'Heat Exchanger',
                       'Mixing Valve',
                       'Source',
                       'Sink',
                       'Source',
                       'Sink']

component_name_list = ['Compressor',
                       'Condenser',
                       'Separator',
                       'Expansion Valve',
                       'Evaporator',
                       'Pump 1',
                       'Cooling Coil',
                       'Mixing Valve 1',
                       'Pump 2',
                       'Heating Coil',
                       'Mixing Valve 2',
                       'Source 1',
                       'Sink 1',
                       'Source 2',
                       'Sink 2']

# defines the solver path of each component
solver_path_list = [directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Compressor/Polynomial based Model',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Condenser/Moving Boundary Model',
                    directory + ' ',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/ExpansionValve',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Evaporator/Moving Boundary Model',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Pump 1',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Cooling Coil/NTU',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Mixing Valve',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Pump 2',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Heating Coil/NTU',
                    directory + '/Cycles/Thermal_Hydraulic_Circuit_V2/Components/Mixing Valve',
                    directory + ' ',
                    directory + ' ',
                    directory + ' ',
                    directory + ' ']

# defines boundary type of each component
component_modeling_type_list = ['PressureBasedComponent',
                                'MassFlowBasedComponent',
                                'SeparatorComponent',
                                'PressureBasedComponent',
                                'MassFlowBasedComponent',
                                'PressureBasedComponent',
                                'MassFlowBasedComponent',
                                'PressureBasedComponent',
                                'PressureBasedComponent',
                                'MassFlowBasedComponent',
                                'PressureBasedComponent',
                                'Source',
                                'Sink',
                                'Source',
                                'Sink']

# defines fluids of each fluid loop
fluid_list = ['R134a', 'INCOMP::TY20', 'Water', 'INCOMP::TY20', 'Water']

circuit = Circuit(jpcm=jpcm,
                  component_type_list=component_type_list,
                  component_name_list=component_name_list,
                  component_modeling_type_list=component_modeling_type_list,
                  solver_path_list=solver_path_list,
                  fluid_list=fluid_list)

Thermal-Hydraulic Circuit has been successfully initialized 

Tearing Variables:
p at Junction 1 and Component Compressor
h at Junction 1 and Component Compressor
p at Junction 2 and Component Compressor
p at Junction 6 and Component Pump 1
h at Junction 6 and Component Pump 1
p at Junction 7 and Component Pump 1
p at Junction 10 and Component Pump 2
h at Junction 10 and Component Pump 2
p at Junction 11 and Component Pump 2
p at Junction 5 and Component Expansion Valve
m at Junction 8 and Component Cooling Coil
m at Junction 12 and Component Heating Coil

Residual Equations:
Mass Flow Balance at Junction 4
Pressure Equality at Junction 1
Enthalpy Equality at Junction 1
Enthalpy Equality at Junction 6
Mass Flow Balance at Junction 8
Mass Flow Balance at Junction 9
Enthalpy Equality at Junction 10
Mass Flow Balance at Junction 12
Mass Flow Balance at Junction 13

Number of design equations necessary to close equation system: 3


In [3]:
# adding component parameter and outputs

# Compressor Frequency f [Hz]
circuit.components['Compressor'].add_component_parameter(name='f',
                                                         value=70)
# Compressor Power P [kW]
circuit.components['Compressor'].add_component_output(name='P')                  

# Condenser Heat Transfer Area [m2]
circuit.components['Condenser'].add_component_parameter(name='A',
                                                        value=8.0)
# Condenser Capacity Q [kW]
circuit.components['Condenser'].add_component_output(name='Q')

# Expansion Valve Flow Coefficient [-]
circuit.components['Expansion Valve'].add_component_parameter(name='CA', 
                                                              value=1.26e-6, 
                                                              scale_factor=1e6,
                                                              is_input=True,
                                                              bounds=(1e-9, 1e5))

# Evaporator Heat Transfer Area [m2]
circuit.components['Evaporator'].add_component_parameter(name='A', value=5.0)

# Evaporator Capacity Q [kW]
circuit.components['Evaporator'].add_component_output(name='Q')

# Pump 1 Flow Characteristic Coefficient [-]
circuit.components['Pump 1'].add_component_parameter(name='k',
                                                     value=0.123,
                                                     scale_factor=1e1,
                                                     is_input=True,
                                                     bounds=(1e-5, 1e5))

# Cooling Coil Overall Heat Transfer Coeffcient [W/m2K]
circuit.components['Cooling Coil'].add_component_parameter(name='UA',
                                                           value=10000.0)
# Cooling Coil Capacity Q [kW]
circuit.components['Cooling Coil'].add_component_output(name='Q')

# Pump 2 Flow Characteristic Coefficient [-]
circuit.components['Pump 2'].add_component_parameter(name='k', 
                                                     value=0.0835,
                                                     scale_factor=1e2,
                                                     is_input=True,
                                                     bounds=(1e-5, 1e5))

# Heating Coil OVerall Heat Transfer Coeffcient [W/m2K]
circuit.components['Heating Coil'].add_component_parameter(name='UA', 
                                                           value=10000.0)
# Cooling Coil Capacity [kW]
circuit.components['Heating Coil'].add_component_output(name='Q')

# Mixing Valve 1 Flow-Coefficient CA [-]
circuit.components['Mixing Valve 1'].add_component_parameter(name='CA', 
                                                             value=0.124, 
                                                             scale_factor=1e1,
                                                             bounds=(1e-5, 1e5))

# Mixing Valve 1 Stroke-Coefficient eps [-]
circuit.components['Mixing Valve 1'].add_component_parameter(name='eps', 
                                                             value=0.006, 
                                                             scale_factor=1e3,
                                                             is_input=True,
                                                             bounds=(0.0, 0.9))


# Mixing Valve 2 Flow-Coefficient CA [-]
circuit.components['Mixing Valve 2'].add_component_parameter(name='CA',
                                                             value=0.08,
                                                             scale_factor=1e2,
                                                             bounds=(1e-5, 1e5))
# Mixing Valve 2 Stroke-Coefficient eps [-]
circuit.components['Mixing Valve 2'].add_component_parameter(name='eps', 
                                                             value=0.9, 
                                                             scale_factor=1.0,
                                                             is_input=True,
                                                             bounds=(0.0, 0.9))

In [4]:
# sets source values

pi_cc_sec = 2.0
Ti_cc_sec = 5.0
mi_cc_sec = 1.0

pi_hc_sec = 1.0
Ti_hc_sec = 30.0
mi_hc_sec = 10.0

circuit.components['Source 1'].parameter['p_source'].set_value(pi_cc_sec * 1e5)
circuit.components['Source 1'].parameter['T_source'].set_value(Ti_cc_sec + 273.15)
circuit.components['Source 1'].parameter['m_source'].set_value(mi_cc_sec)
circuit.components['Source 1'].parameter['m_source'].is_input = True
circuit.components['Source 1'].parameter['m_source'].initial_value = mi_cc_sec
circuit.components['Source 1'].parameter['m_source'].bounds = (0.01, 1000.0)

circuit.components['Source 2'].parameter['p_source'].set_value(pi_hc_sec * 1e5)
circuit.components['Source 2'].parameter['T_source'].set_value(Ti_hc_sec + 273.15)
circuit.components['Source 2'].parameter['m_source'].set_value(mi_hc_sec)
circuit.components['Source 2'].parameter['m_source'].is_input = True
circuit.components['Source 2'].parameter['m_source'].initial_value = mi_hc_sec
circuit.components['Source 2'].parameter['m_source'].bounds = (0.01, 10000.0)

In [5]:
# adds and sets design criteria value
SH = 4.5
SC = 2.0
To_cc = Ti_cc_sec - 5.0

# adds design equations to circuit
circuit.add_design_equa(SuperheatEquation(circuit.components['Evaporator'], SH, 'out', 'h', psd['-c'], relaxed=True))
# circuit.add_design_equa(SubcoolingEquation(circuit.components['Condenser'], SC, 'out', 'h', psd['-h'], relaxed=True))
# circuit.add_design_equa(DesignParameterEquation(circuit.components['Cooling Coil'], To_cc + 273.15, 'out', 'T', psd['-c'], relaxed=True))


In [6]:
# solver initial values
p1_1 = PropsSI('P', 'T', Ti_cc_sec + 273.15 - 5.0, 'Q', 1.0, fluid_list[0])
h1_1 = 4.0e5
p2_1 = PropsSI('P', 'T', Ti_hc_sec + 273.15 + 5.0, 'Q', 1.0, fluid_list[0])
p6_6 = 1.0e5
h6_6 = PropsSI('H', 'T', Ti_cc_sec + 273.15, 'P', p6_6, fluid_list[1])
p7_6 = 1.1e5
p10_9 = 1.0e5
h10_9 = PropsSI('H', 'T', Ti_hc_sec + 273.15, 'P', p10_9, fluid_list[2])
p11_9 = 1.15e5
p5_4 = p1_1
m8_7 = 0.1
m12_0 = 0.1

x0 = [p1_1, h1_1, p2_1, p6_6, h6_6, p7_6, p10_9, h10_9, p11_9, p5_4, m8_7, m12_0]

# tearing variable bounds
Vt_bnds = [(0.1e5, 5e5),
           (3e5, 6e5),
           (5.1e5, 30e5),
           (0.1e5, 1e5),
           (-5e4, -1e4),
           (1.01e5, 10e5),
           (0.1e5, 1e5),
           (1e4, 1e6),
           (1.01e5, 10e5),
           (0.1e5, 5.0e5),
           (0.000001, 100.0),
           (0.000001, 100.0)]

# sets initial values and bounds of tearing variables
for i, var in enumerate(circuit.Vt):
    var.initial_value = x0[i]
    var.bounds = Vt_bnds[i]

In [7]:
    # runs the system solver
    sol = system_solver(circuit)

  NIT    FC           OBJFUN            GNORM
    1     2     1.895963E-02     0.000000E+00
Condenser failed!


RuntimeError: Tried to solve equation: Enthalpy Equality, but it is not solvable yet.

In [ ]:
    # solution vector of tearing variables
    x = sol['x']

    if not sol['converged']:
        print(sol['message'])

    else:

        circuit.solve(x[0:len(circuit.Vt)])

        # junction values corresponding to the solution
        pi_co, hi_co, mi_co = circuit.components['Compressor'].ports[psd['p']].p.value, circuit.components['Compressor'].ports[psd['p']].h.value, circuit.components['Compressor'].ports[psd['p']].m.value
        pi_c, hi_c, mi_c = circuit.components['Condenser'].ports[psd['h']].p.value, circuit.components['Condenser'].ports[psd['h']].h.value, circuit.components['Condenser'].ports[psd['h']].m.value
        pi_se, hi_se, mi_se = circuit.components['Separator'].ports[psd['tp']].p.value, circuit.components['Separator'].ports[psd['tp']].h.value, circuit.components['Separator'].ports[psd['tp']].m.value
        pi_ev, hi_ev, mi_ev = circuit.components['Expansion Valve'].ports[psd['p']].p.value, circuit.components['Expansion Valve'].ports[psd['p']].h.value, circuit.components['Expansion Valve'].ports[psd['p']].m.value
        pi_v, hi_v, mi_v = circuit.components['Evaporator'].ports[psd['c']].p.value, circuit.components['Evaporator'].ports[psd['c']].h.value, circuit.components['Evaporator'].ports[psd['c']].m.value

        # plots log ph diagramm
        logph([[hi_co * 1e-3, hi_c * 1e-3, hi_se * 1e-3, hi_ev * 1e-3, hi_v * 1e-3, hi_co * 1e-3]],
              [[pi_co * 1e-5, pi_c * 1e-5, pi_se * 1e-5, pi_ev * 1e-5, pi_v * 1e-5, pi_co * 1e-5]],
              [[1, 2, 3, 4, 5, 1]],
              [fluid_list[0]])

        for key in circuit.components:
            print(f'{circuit.components[key].name}:')
            for port in circuit.components[key].ports:
                T = PropsSI('T', 'H', circuit.components[key].ports[port].h.value, 'P', circuit.components[key].ports[port].p.value, circuit.components[key].ports[port].fluid) - 273.15
                print(f'Port ID {circuit.components[key].ports[port].port_id}: '
                      f'p = {round(circuit.components[key].ports[port].p.value * 1e-5, 3)} bar, '
                      f'T = {round(T, 3)} °C, '
                      f'h = {round(circuit.components[key].ports[port].h.value * 1e-3, 3)} kJ/kg, '
                      f'm = {round(circuit.components[key].ports[port].m.value, 3)} kg/s')
            for param in circuit.components[key].parameter:
                print(f'{param}: {round(circuit.components[key].parameter[param].value, 3)}')
            print('\n')
